[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/seagarwu/mynotebooks/blob/main/shioaji/shioaji_onlinedoc/shioaji-order-stock.ipynb)

# Stock

## Place Order
[Stock place order jupyter link](https://nbviewer.jupyter.org/github/Sinotrade/Sinotrade.github.io/blob/master/tutorial/stock.ipynb)

### First, you need to [login](shioaji-quick_start.ipynb) and [activate CA](shioaji-order-ca_signature.ipynb).

In [8]:
import shioaji as sj
import getpass as gp

api = sj.Shioaji()
user_id, user_pass = gp.getpass("USER ID: "), gp.getpass("USER PASS: ")
accounts = api.login(user_id, user_pass)

Response Code: 0 | Event Code: 0 | Info: host '203.66.91.161:80', hostname '203.66.91.161:80' IP 203.66.91.161:80 (host 1 of 1) (host connection attempt 1 of 1) (total connection attempt 1 of 1) | Event: Session up


In [9]:
import tkinter as tk
from tkinter import filedialog

root = tk.Tk()
root.withdraw()
file_path = filedialog.askopenfilename()

api.activate_ca(
    ca_path = file_path,
    ca_passwd = user_pass,
    person_id = user_id,
)

True

### Get Contract Object.

In [11]:
contract = api.Contracts.Stocks.TSE.TSE2330
contract

Stock(exchange=<Exchange.TSE: 'TSE'>, code='2330', symbol='TSE2330', name='台積電', category='24', unit=1000, limit_up=578.0, limit_down=473.5, reference=526.0, update_date='2022/04/28', margin_trading_balance=1368, short_selling_balance=308, day_trade=<DayTrade.Yes: 'Yes'>)

### Making Order Object.

在發送委託單前，要先產生一個Order物件。<br>
Order物件建立的參數說明如下：

|參數|參數說明|參數範例|
|:----|:----|:----|
|price|委託價格|18.5|
|quantity|委託數量|1|
|action|委託單動作|{Buy, Sell}|
|price_type|價格類型|{LMT, MKT, MKP} (限價、市價、範圍市價)|
|order_type|委託單類型|{ROD, IOC, FOK} (當日有效、立即成交否則取消、全部成交否則取消)|
|order_cond|委託單種類|{Cash, MarginTrading, ShortSelling} (現股、融資、融券)|
|order_lot|委託單交易單位|{Common, Fixing, Odd, IntradayOdd} (整股、盤後定價、盤後零股、盤中零股)|
|first_sell|是否為現沖先賣|{true, false}|
|octype|倉別|{Auto, NewPosition, Cover, DayTrade} (自動、新倉、平倉、當沖)|
|OptionRight|選擇權類別|{Call, Put}|
|account|交易帳戶|可由API取得account物件|

> <br>order_cond、order_lot及first_sell，為股票Order物件特有屬性<br>
> octype，為期貨或選擇權Order物件特有屬性<br>
> OptionRight為選擇權Order物件特有屬性

#### Buy 現股買進

In [10]:
order = api.Order(
    price=12, 
    quantity=1, 
    action=sj.constant.Action.Buy,  # 現股買進
    price_type=sj.constant.StockPriceType.LMT, 
    order_type=sj.constant.TFTOrderType.ROD, 
    order_lot=sj.constant.TFTStockOrderLot.Common, 
    account=api.stock_account
)
order

Order(action=<Action.Buy: 'Buy'>, price=12, quantity=1, price_type=<StockPriceType.LMT: 'LMT'>, order_type=<FuturesOrderType.ROD: 'ROD'>)

#### Sell 現股賣出

In [7]:
order = api.Order(
    price=12, 
    quantity=1, 
    action=sj.constant.Action.Sell,  # 現股賣出
    price_type=sj.constant.StockPriceType.LMT, 
    order_type=sj.constant.TFTOrderType.ROD, 
    order_lot=sj.constant.TFTStockOrderLot.Common, 
    account=api.stock_account
)
order

Order(action=<Action.Sell: 'Sell'>, price=12, quantity=1, price_type=<StockPriceType.LMT: 'LMT'>, order_type=<FuturesOrderType.ROD: 'ROD'>)

#### First Sell 現沖先賣

In [8]:
order = api.Order(
    price=12, 
    quantity=1, 
    action=sj.constant.Action.Sell,
    price_type=sj.constant.StockPriceType.LMT,
    order_type=sj.constant.TFTOrderType.ROD,
    order_lot=sj.constant.TFTStockOrderLot.Common,
    first_sell=sj.constant.StockFirstSell.Yes,  #現沖先賣，設定為StockFirstSell.Yes or True
    account=api.stock_account
)
order

Order(action=<Action.Sell: 'Sell'>, price=12, quantity=1, price_type=<StockPriceType.LMT: 'LMT'>, order_type=<FuturesOrderType.ROD: 'ROD'>, first_sell=<StockFirstSell.Yes: 'true'>)

#### 盤中零股買進

In [3]:
order = api.Order(
    price=12, 
    quantity=1, 
    action=sj.constant.Action.Buy, #買進
    price_type=sj.constant.StockPriceType.LMT, 
    order_type=sj.constant.TFTOrderType.ROD, 
    order_lot=sj.constant.TFTStockOrderLot.IntradayOdd, #指定盤中零股
    account=api.stock_account
)
order

Order(action=<Action.Buy: 'Buy'>, price=12, quantity=1, price_type=<StockPriceType.LMT: 'LMT'>, order_type=<FuturesOrderType.ROD: 'ROD'>, order_lot=<TFTStockOrderLot.IntradayOdd: 'IntradayOdd'>)

#### 盤後定價買進

In [4]:
order = api.Order(
    price=12, 
    quantity=1, 
    action=sj.constant.Action.Buy, #買進
    price_type=sj.constant.StockPriceType.LMT, 
    order_type=sj.constant.TFTOrderType.ROD, 
    order_lot=sj.constant.TFTStockOrderLot.Fixing, #指定盤後定價
    account=api.stock_account
)
order

Order(action=<Action.Buy: 'Buy'>, price=12, quantity=1, price_type=<StockPriceType.LMT: 'LMT'>, order_type=<FuturesOrderType.ROD: 'ROD'>, order_lot=<TFTStockOrderLot.Fixing: 'Fixing'>)

#### 盤後零股買進

In [5]:
order = api.Order(
    price=12, 
    quantity=1, 
    action=sj.constant.Action.Buy, #買進
    price_type=sj.constant.StockPriceType.LMT, 
    order_type=sj.constant.TFTOrderType.ROD, 
    order_lot=sj.constant.TFTStockOrderLot.Odd, #指定盤後零股
    account=api.stock_account
)
order

Order(action=<Action.Buy: 'Buy'>, price=12, quantity=1, price_type=<StockPriceType.LMT: 'LMT'>, order_type=<FuturesOrderType.ROD: 'ROD'>, order_lot=<TFTStockOrderLot.Odd: 'Odd'>)

以上為現股Order的建立及下單相關操作，若是要使用融資或融券，只要在建立Order時指定order_cond為StockOrderCond.MarginTrading(融資)或是StockOrderCond.ShortSelling(融券)即可；若在建立Order時沒有指定order_cond，預設都是以StockOrderCond.Cash建立。

#### Order Object

> <details><summary>Attributes:</summary>
>
> price (float or int): the price of order<br>
> quantity (int): the quantity of order<br>
> action (str): order action to buy or sell<br>
>     {Buy, Sell}<br>
> price_type (str): pricing type of order<br>
>     {LMT, MKT, MKP} (限價、市價、範圍市價)<br>
> order_type (str): the type of order<br>
>     {ROD, IOC, FOK}<br>
> order_cond (str): order condition stock only<br>
>     {Cash, MarginTrading, ShortSelling} (現股、融資、融券)<br>
> order_lot (str): the type of order<br>
>     {Common, Fixing, Odd, IntradayOdd} (整股、定盤、盤後零股、盤中零股)<br>
> first_sell {str}: the type of first sell<br>
>     {true, false}<br>
> account (:obj:Account): which account to place this order<br>
> ca (binary): the ca of this order<br>
</details>

### Place Order

In [ ]:
# 會實際下單(證券)，請小心! (但我沒有證券帳號，所以下單會error!)
trade = api.place_order(contract, order)
trade

## For Example

### Place Order ROD LMT

In [ ]:
# 會實際下單(證券)，請小心! (但我沒有證券帳號，所以下單會error!)
contract = api.Contracts.Stocks.TSE.TSE2330
order = api.Order(
    price=12,
    quantity=1,
    action=sj.constant.Action.Buy,
    price_type=sj.constant.StockPriceType.LMT,
    order_type=sj.constant.TFTOrderType.ROD,
    account=api.stock_account
)
trade = api.place_order(contract, order)
trade

### Place Order about First Sell

In [ ]:
# 會實際下單(證券)，請小心! (但我沒有證券帳號，所以下單會error!)
contract = api.Contracts.Stocks.TSE.TSE2330
order = api.Order(
    price=14,
    quantity=1,
    action=sj.constant.Action.Sell,
    price_type=sj.constant.StockPriceType.LMT,
    order_type=sj.constant.TFTOrderType.ROD,
    first_sell=sj.constant.StockFirstSell.Yes,
    account=api.stock_account
)
trade = api.place_order(contract, order)
trade

> **Status of Trade**<br>

> PendingSubmit: 傳送中<br>
> PreSubmitted: 預約單<br>
> Submitted: 傳送成功<br>
> Failed: 失敗<br>
> Cancelled: 已刪除<br>
> Filled: 完全成交<br>
> Filling: 部分成交<br>

### Cancel Order

In [ ]:
api.update_status(api.stock_account)
api.cancel_order(trade)
api.update_status(api.stock_account)
trade

OrderState.FOrderOrderState.FOrder {'operation': {'op_type': 'New', 'op_code': '99QB', 'op_msg': '超過交易額度     0萬,請至E櫃檯或易利得3380調整'}, 'order': {'id': '1f92361d', 'seqno': '331991', 'ordno': '00', 'account': {'account_type': 'F', 'person_id': '', 'broker_id': 'F002002', 'account_id': '8070913', 'signed': True}, 'action': 'Buy', 'price': 10200.0, 'quantity': 1, 'order_type': 'ROD', 'price_type': 'LMT', 'market_type': 'Night', 'oc_type': 'New', 'subaccount': '', 'combo': False}, 'status': {'id': '1f92361d', 'exchange_ts': 1651080172, 'modified_price': 0.0, 'cancel_quantity': 0, 'order_quantity': 1, 'web_id': 'Z'}, 'contract': {'security_type': 'FUT', 'code': 'MXF', 'exchange': 'TIM', 'delivery_month': '202205', 'delivery_date': '', 'strike_price': 0.0, 'option_right': 'Future'}}
 {'operation': {'op_type': 'New', 'op_code': '99QB', 'op_msg': '超過交易額度     0萬,請至E櫃檯或易利得3380調整'}, 'order': {'id': '1f92361d', 'seqno': '331991', 'ordno': '00', 'account': {'account_type': 'F', 'person_id': '', 'broker

### Update Order

In [ ]:
api.update_status(api.stock_account)
api.update_order(trade=trade, price=12, qty=2)
api.update_status(api.stock_account)
trade

---
### Prev: [Order / CA Signature](shioaji-order-ca_signature.ipynb) Next: [Order / Future and Option](shioaji-order-future_and_option.ipynb)